# Content planner flow

Before you get started:

Create a .env file in the directory and store these two values:

- FIRECRAWL_API_KEY="fc-..." (get the firecrawl API key here: https://www.firecrawl.dev/i/api)
- OPENAI_API_KEY="sk-..."

In [20]:
#!pip install firecrawl-py

In [21]:
# Importing necessary libraries
import os
import uuid
import yaml
import json
from pathlib import Path
from pydantic import BaseModel
from typing import Optional

# Firecrawl SDK
from firecrawl import FirecrawlApp

# Importing Crew related components
from crewai import Agent, Task, Crew, LLM

# Importing CrewAI Flow related components
from crewai.flow.flow import Flow, listen, start, router, or_

from dotenv import load_dotenv
load_dotenv()

import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os

# Load environment variables
load_dotenv()

# Option D: Azure OpenAI
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

llm = LLM(
    model="azure/gpt-4o-mini",
    api_key=openai_api_key,
    base_url=openai_endpoint,
    api_version=openai_api_version,
    azure=True
)

print("🚀 Environment configured!")
print(f"LLM Model: {llm.model}")

In [23]:
blog_post_url = "https://blog.dailydoseofds.com/p/5-chunking-strategies-for-rag"

## Twitter and LinkedIn Planning Crew

In [24]:
# define structured output for twitter and linkedin

class Tweet(BaseModel):
    """Represents an individual tweet in a thread"""
    content: str
    is_hook: bool = False  # Identifies if this is the opening/hook tweet
    media_urls: Optional[list[str]] = []  # Optional media attachments (images, code snippets)

class Thread(BaseModel):
    """Represents a Twitter thread"""
    topic: str  # Main topic/subject of the thread
    tweets: list[Tweet]  # List of tweets in the thread

class LinkedInPost(BaseModel):
    """Represents a LinkedIn post"""
    content: str
    media_url: str # Main image url for the post

In [ ]:
# load agent and task configurations from yaml files and tools

from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
)

# Load agent and task configurations from YAML files
with open('config/planner_agents.yaml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('config/planner_tasks.yaml', 'r') as f:
    tasks_config = yaml.safe_load(f)

In [6]:
# create agents, their tasks and crew for twitter

draft_analyzer = Agent(config=agents_config['draft_analyzer'], tools=[
    DirectoryReadTool(),
    FileReadTool()
], llm=llm)

twitter_thread_planner = Agent(config=agents_config['twitter_thread_planner'], tools=[
    DirectoryReadTool(),
    FileReadTool()
], llm=llm)

analyze_draft = Task(
  config=tasks_config['analyze_draft'],
  agent=draft_analyzer
)

create_twitter_thread_plan = Task(
  config=tasks_config['create_twitter_thread_plan'],
  agent=twitter_thread_planner,
  output_pydantic=Thread
)

twitter_planning_crew = Crew(
    agents=[draft_analyzer, twitter_thread_planner],
    tasks=[analyze_draft, create_twitter_thread_plan],
    verbose=False
)

In [7]:
# create agents, their tasks and crew for linkedin

linkedin_post_planner = Agent(config=agents_config['linkedin_post_planner'], tools=[
    DirectoryReadTool(),
    FileReadTool()
    ], llm=llm)

create_linkedin_post_plan = Task(
  config=tasks_config['create_linkedin_post_plan'],
  agent=linkedin_post_planner,
  output_pydantic=LinkedInPost
)

linkedin_planning_crew = Crew(
    agents=[draft_analyzer, linkedin_post_planner],
    tasks=[analyze_draft, create_linkedin_post_plan],
    verbose=False
)

In [ ]:
key = os.getenv("FIRECRAWL_API_KEY")
print(key)

import firecrawl
print(firecrawl.__version__)

In [9]:
# define state for the content planning flow

class ContentPlanningState(BaseModel):
    """
    State for the content planning flow
    """
    blog_post_url: str = blog_post_url
    draft_path: Path = "assets/ "
    post_type: str = "twitter"
    path_to_example_threads: str = "assets/example_threads.txt"
    path_to_example_linkedin: str = "assets/example_linkedin.txt"

class CreateContentPlanningFlow(Flow[ContentPlanningState]):
    # No need for AI Agents on this step, so we just use regular Python code
    @start()
    def scrape_blog_post(self):
        print(f"# fetching draft from: {self.state.blog_post_url}")
        app = FirecrawlApp(api_key=os.getenv("FIRECRAWL_API_KEY"))
        scrape_result = app.scrape_url(url=self.state.blog_post_url, formats= [ 'markdown','html' ])
        try:
          print(scrape_result)
          title = scrape_result.metadata.get('title', 'No title')
          print(f"# Title: {title}")
        except Exception as e:
          title = str(uuid.uuid4())
        self.state.draft_path = f'assets/{title}.md'
        with open(self.state.draft_path, 'w') as f:
          f.write(scrape_result.markdown)
        return self.state

    @router(scrape_blog_post)
    def select_platform(self):
        if self.state.post_type == "twitter":
          return "twitter"
        elif self.state.post_type == "linkedin":
          return "linkedin"

    @listen("twitter")
    def twitter_draft(self):
        print(f"# Planning content for: {self.state.draft_path}")
        
        result = twitter_planning_crew.kickoff(inputs={'draft_path': self.state.draft_path, 'path_to_example_threads': self.state.path_to_example_threads})
        
        print(f"# Planned content for {self.state.draft_path}:")
        
        for i, tweet in enumerate(result.pydantic.tweets):
            
            print(f"Tweet {i+1}:")
            print(f"{tweet.content}")
            print(f"Media URLs: {tweet.media_urls}")

            print("-"*100)
        return result
    
    @listen("linkedin")
    def linkedin_draft(self):
        print(f"# Planning content for: {self.state.draft_path}")
        result = linkedin_planning_crew.kickoff(inputs={'draft_path': self.state.draft_path, 'path_to_example_linkedin': self.state.path_to_example_linkedin})
        print(f"# Planned content for {self.state.draft_path}:")
        print(f"{result.pydantic.content}")
        return result

    @listen(or_(twitter_draft, linkedin_draft))
    def save_plan(self, plan):
        with open(f'output/{self.state.draft_path.split("/")[-1]}_{self.state.post_type}.json', 'w') as f:
            json.dump(plan.pydantic.model_dump(), f, indent=2)

In [ ]:
# Plot the flow
flow = CreateContentPlanningFlow()
flow.state.post_type = "twitter"

In [ ]:
flow.plot()

In [ ]:
flow.state

In [ ]:
flow.kickoff()

In [ ]:
flow.state.post_type = "linkedin"
flow.kickoff()